In [64]:
#The purpose of this file is to look at during what season (on average) a certain crime tends to peak in activity (in a certain county)
#I'll then go and apply some k means to it or something to try and find common structure
# we can still use k means clustering to find common patterns in qualitative data as long as i encode the data first
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [65]:
main_df

,Quarter,Garda Division,Type of Offence,VALUE
5,2003Q1,Cavan-Monaghan Garda Division,"Attempts/threats to murder, assaults, harassme...",128.0
8,2003Q1,Cavan-Monaghan Garda Division,Burglary and related offences,136.0
10,2003Q1,Cavan-Monaghan Garda Division,Controlled drug offences,26.0
13,2003Q1,Cavan-Monaghan Garda Division,Damage to property and to the environment off...,145.0
16,2003Q1,Cavan-Monaghan Garda Division,Dangerous or negligent acts,160.0
...,...,...,...,...
216561,2025Q3,Wicklow Garda Division,Road and traffic offences,184.0
216567,2025Q3,Wicklow Garda Division,"Robbery, extortion and hijacking offences",7.0
216568,2025Q3,Wicklow Garda Division,Sexual offences,18.0
216571,2025Q3,Wicklow Garda Division,Theft and related offences,465.0


In [66]:
main_df = pd.read_csv('CJQ06.20260202004354.csv')
print(main_df.columns)
main_df = main_df.drop(["STATISTIC","C02480V03003","C02481V03160","TLIST(Q1)","UNIT"],axis=1)
pattern = r'\((\d{2})\)'
main_df = main_df.groupby(
    ["Quarter", "Garda Division", "Type of Offence"],  
    as_index=False  
)["VALUE"].sum()
main_df = main_df.loc[main_df["Type of Offence"].str.contains(pattern, regex=True)]




Index(['STATISTIC', 'Statistic Label', 'TLIST(Q1)', 'Quarter', 'C02481V03160',
       'Garda Division', 'C02480V03003', 'Type of Offence', 'UNIT', 'VALUE'],
      dtype='str')


C:\Users\downe\AppData\Local\Temp\ipykernel_16028\653793310.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  main_df = main_df.loc[main_df["Type of Offence"].str.contains(pattern, regex=True)]


In [67]:
main_df["Type of Offence"] = main_df["Type of Offence"].str.replace(r"\s\(\d{2}\)$", "", regex=True)
crimes = main_df["Type of Offence"].unique()


In [68]:
unique_division = main_df['Garda Division'].unique().tolist()
#replacing "/" with "-" to make writing filepaths easier
unique_division = [div.replace('/', '-') for div in unique_division]
main_df['Garda Division'] = main_df['Garda Division'].str.replace('/', '-')
main_df.to_csv("cleaned_quarters_df.csv",index=False)

In [69]:
main_df

,Quarter,Garda Division,Type of Offence,VALUE
5,2003Q1,Cavan-Monaghan Garda Division,"Attempts/threats to murder, assaults, harassme...",128.0
8,2003Q1,Cavan-Monaghan Garda Division,Burglary and related offences,136.0
10,2003Q1,Cavan-Monaghan Garda Division,Controlled drug offences,26.0
13,2003Q1,Cavan-Monaghan Garda Division,Damage to property and to the environment off...,145.0
16,2003Q1,Cavan-Monaghan Garda Division,Dangerous or negligent acts,160.0
...,...,...,...,...
216561,2025Q3,Wicklow Garda Division,Road and traffic offences,184.0
216567,2025Q3,Wicklow Garda Division,"Robbery, extortion and hijacking offences",7.0
216568,2025Q3,Wicklow Garda Division,Sexual offences,18.0
216571,2025Q3,Wicklow Garda Division,Theft and related offences,465.0


In [70]:
sub_dfs = {} # key = garda division, value = dataframe
for division in unique_division:
    sub_df = main_df.loc[main_df["Garda Division"]==division]
    sub_dfs[division] = sub_df

In [71]:
#writing each sub_df to its own individual csv file for future reference and ease
curr_dir = os.getcwd()
if not os.path.exists(os.path.join(curr_dir,"counties")):
    os.mkdir(os.path.join(curr_dir,"counties"),mode=0o755)

for division in sub_dfs.keys():
    division_path = os.path.join(curr_dir, "counties", division)
    os.makedirs(division_path, exist_ok=True)

    file_path = os.path.join(division_path, f"{division}_quarters_crime.csv")
    sub_dfs[division].to_csv(file_path, index=False)

In [72]:
dfs = []
for file in os.listdir("counties"):
    dfs.append(pd.read_csv(os.path.join(f"counties\\{file}",f"{file}_quarters_crime.csv")))
    